# 3. 陸域保護區面積(公告)

https://gist.github.com/mutolisp/7feecd5ee30da9999b807391310a90a0

## 資料來源及 metadata

* 原始資料來源：行政院農委會林務局
* 原始資料格式：Microsoft Excel xml format 2007 (xlsx)

## 目標

* 計算出各保護區的面積
* 各保護區的年度面積變化(保護區面積-年度)


## 預先安裝

* [LibreOffice](http://libreoffice.org)
* [R project](http://www.r-project.org)
    * [data.table](https://cran.r-project.org/web/packages/data.table)
    * [stringr](https://cran.r-project.org/web/packages/stringr)

## 資料清理

「資料清理(data cleaning)」這個階段依照不同類型的資料具有不同的作法，依照 Maletic & Marcus (2000) 所提出的原則，資料清理是一個過程，過程中可分為三個階段：

 1. 定義並決定錯誤的類型
 2. 搜尋並辨識出錯誤的實證(error instances)
 3. 更正未被找到的錯誤

如果是跟空間圖資有關係的，可以使用圖來輔助找出錯誤

### 階段一、篩選（screening，定義並決定有哪些錯誤的類型）

在資料清理的第一個階段中，對於一個試算表類(spreadsheet，或是 table)的資料所而言，
通常會先看資料的原始格式、欄位定義、數值大小等。對某些的使用者來說，有些人會在試算表中設定文字格式、
定義欄位中的資料類型(例如：數值(numeric)、浮點數(floating number)、文字(character)等)、
數值精確度等，所以在篩選的階段我們必須先將這些資訊查閱清楚，並快速判斷資料是否有錯誤，
以及資料可能錯誤的類型等。對於篩選的第一步驟，先假定資料在專業知識(對專家而言)或一般常識下，
資料的樣貌(即上面所提到的欄位定義、文字格式等)和產製可能會怎麼做？如果是你該怎麼進行？

#### 0. 資料可能的樣貌

在還沒看到資料前，我們預先思考若是我們自己要去產製出臺灣陸域保護區的面積時，該如何做？
資料中欄位的定義、類型、數值範圍為何？有些資料的產製和該領域專業知識(domain knowledge)有關，
有些則是和技術處理有關。對於陸域保護區面積的資料，如果是試算表或文字形式的資料，
很直覺想到的一定會有**保護區的名稱**和**保護區的面積**。接下來要思考面積是怎麼計算出來的？
如果是量測的，那就得去找是哪些機關單位量測的，量測的方法是什麼。如果是推算的，
例如使用地理資訊系統(GIS)來計算保護區幾何多邊形的面積。先有一些概念後，
接下來則是直接來看原始資料是不是和我們預想的相符，如果不是我們所想的，
差異性在哪？因為不同的資料會有不同的處理方法。

#### 1. 資料轉換與輸入

在這個步驟中，我們先將 xlsx 的資料轉成 csv，xlsx 可以使用 xlsx library 來讀，或是先使用 LibreOffice 來轉成 csv，再用 R 的```read.table()``` 或 ```read.csv()``` 來讀取。如果使用 LibreOffice，可以用 command line 轉換為 csv ([Libre Office Filter Option 參考網頁](https://wiki.openoffice.org/wiki/Documentation/DevGuide/Spreadsheets/Filter_Options)):

```sh
/Applications/LibreOffice.app/Contents/MacOS/soffice 
    --headless --infilter="Calc MS Excel 2007 XML:UTF-8" 
    --convert-to csv:"Text - txt - csv (StarCalc)":"124,34,0,1,,0" tw_national_protection_area_raw.xlsx
```
(*註： 這個是 MacOS X 底下的使用， 分隔符號採用 "|")

輸入進 R 來看：


In [1]:
library(data.table)
# jupyter data.table 有些問題，所以更新 DT 時會列出整個表，為了解決這個問題
# 我們使用 invisible() 來隱藏列表

options(datatable.prettyprint.char=5L)
# 讀進 R ，並將資料表命名為 np_raw 。
# 一般 csv 是用逗號分隔，因為看了資料表欄位資料中有逗號，所以在上面轉換成 csv 的過程中
# 我們使用 pipeline (|) 來分隔
# 讀取資料通常使用 read.table() 或 read.csv()，這裡我們用 data.table 中的 fread 來讀取
np_raw <- fread('data/tw_national_protection_area_raw.csv', header=TRUE, sep = "|")

接下來用 ```head()``` 看資料表的前幾筆記錄，看是否正確讀對檔案、編碼是否有錯誤(UTF-8 或 Big5)、欄位的類型是什麼

In [2]:
head(np_raw)

保護區名稱,面積(公頃),範圍(位置),管理機關,公告日期
淡水河紅樹林自然保留區,76.41,新北市竹圍附近淡水河沿岸風景保安林,行政院農業委員會林務局羅東林區管理處(國定自然地景),75.06.27
關渡自然保留區,55,臺北市關渡堤防外沼澤區,臺北市政府(國定自然自然地景),75.06.27
坪林台灣油杉自然保留區,34.6,羅東林區管理處文山事業區第 28、29、40、41林班,羅東林區管理處(國定自然地景),75.06.27
哈盆自然保留區,332.7,宜蘭縣員山鄉宜蘭事業區第57林班，新北市烏來區烏來事業區第72、15林班,行政院農業委會林業試驗所福山研究中心(國定自然地景),75.06.27
插天山自然保留區,7759.17,大溪事業區部分：第13-15、24-26、32林班及第33林班中扣除已開發經營面積75公頃達觀山自然保護區之範圍；烏來事業區部分:第18、41-45、49-53林班及第35林班扣除滿月圓森林遊樂區用地850.22公頃之範圍,行政院農業委員會林務局新竹林區管理處(國定自然地景),81.03.12
鴛鴦湖自然保留區,374,大溪事業區第90、91、89林班,行政院國軍退除役官兵輔導委員會森林保育處(國定自然地景),75.06.27


我們在這個步驟很快速得知資料的欄位有「保護區名稱」、「面積(公頃)」、「範圍(位置)」、「管理機關」及「公告日期」五個欄位。接下來更進一步檢查資料欄位中的定義、類型及完整性

#### 2. 檢查欄位定義、類型及完整性

所以我們將檢查一些基本資訊，例如：

* 資料表的維度(欄、列)
* 欄位的定義
* 欄位的資料類型、精確度以及一致性


In [3]:
# 使用 dim() 來看資料的筆數及欄位數目
dim(np_raw)

[1] 95  5

從 dim 的結果可以知道總資料筆數為 95 筆和 5 個欄位。依照原始資料的欄位，我們先整理出其定義及資料類型，因為資料處理上的方便，我們將中文欄位名稱改成具有意義的英文或縮寫，同時也把特殊符號(例如空格、點、dash 等)去除，並初步判斷其資料類型(data types)，整理如下表：

| 欄位名稱    | 英文或縮寫      | 資料類型         | 備註     |
| ---------- | ------------  | --------------  | ------- |
| 保護區名稱  | np_name        | 文字(character) |          |
| 面積(公頃)  | area          |  浮點數(float)   |            |
| 範圍(位置)  | location      |  文字(character) |            |
| 管理機關    | admin_agency  |  文字(character) |            |
| 公告日期    | announce_date |  日期(date)      | 民國年.月.日 |


In [4]:
# 使用 colnames() 來重新命名欄位名稱
colnames(np_raw) = c('np_name', 'area', 'location', 'admin_agency', 'announce_date')
# 再看一下資料本身有沒有問題
head(np_raw)

np_name,area,location,admin_agency,announce_date
淡水河紅樹林自然保留區,76.41,新北市竹圍附近淡水河沿岸風景保安林,行政院農業委員會林務局羅東林區管理處(國定自然地景),75.06.27
關渡自然保留區,55,臺北市關渡堤防外沼澤區,臺北市政府(國定自然自然地景),75.06.27
坪林台灣油杉自然保留區,34.6,羅東林區管理處文山事業區第 28、29、40、41林班,羅東林區管理處(國定自然地景),75.06.27
哈盆自然保留區,332.7,宜蘭縣員山鄉宜蘭事業區第57林班，新北市烏來區烏來事業區第72、15林班,行政院農業委會林業試驗所福山研究中心(國定自然地景),75.06.27
插天山自然保留區,7759.17,大溪事業區部分：第13-15、24-26、32林班及第33林班中扣除已開發經營面積75公頃達觀山自然保護區之範圍；烏來事業區部分:第18、41-45、49-53林班及第35林班扣除滿月圓森林遊樂區用地850.22公頃之範圍,行政院農業委員會林務局新竹林區管理處(國定自然地景),81.03.12
鴛鴦湖自然保留區,374,大溪事業區第90、91、89林班,行政院國軍退除役官兵輔導委員會森林保育處(國定自然地景),75.06.27


In [5]:
# 設定 primary key (唯一識別碼)
setkey(np_raw, np_name)

### 階段二、搜尋並辨識出錯誤的實證

設定完欄位名稱、初步看過資料表後，顯示臺灣的保護區系統總共有 95 個保護區，接下來逐一檢查資料的欄位及內容。根據 Maletic & Marcus (2000) 的原則，我們將先初步定義錯誤的類型：

a. **同一欄位具有兩種以上不同屬性的資料**

   例如：在資料欄位中的「管理機關(admin_agency)」，照字面上的意義應該是管理該保護區系統的主管機關，但資料中卻出現括號附註國定自然地景
   
   <div style="background-color: #86EAF4; padding: 10px; margin: 5px">
   行政院農業委員會林務局羅東林區管理處(國定自然地景)
   </div>
   
   這類明顯是不同屬性的資料，應該分拆成不同欄位

b. **數值資料或文字錯誤**

   例如：第二筆資料中的「管理機關」，後面標註的「國定自然自然地景」，「自然自然」明顯為錯誤

c. **具有關聯性的資料錯誤**

   這個類型需要較長時間的檢查，或是從外部資料交叉比對，例如自然保護區是從林班來界定的，則去取得林班面積表或相關圖資，套用看是否與公布資料一致。或是像保護區有地名的和地點位置有出入，則要再次確認是否正確。
   
d. **空間上重疊的保護區面積**

   因為空間上不同的保護區可能會重疊，因此我們在計算保護區面積時必須扣除重疊的面積

定義完錯誤類型後，我們開始快速篩選資料，搜尋是否有符合上述錯誤定義的實例。

**逐一檢查欄位**

在這個階段，我們將針對欄位(column，或稱為 variables)個別進行檢查。第一個檢查的是欄位中是否有不同屬性的資料，接下來看內容是否有數值或文字錯誤，最後在檢查是否有關聯性的資料錯誤

*1. 保護區名稱*

In [6]:
# 保護區名稱
# 再看一次資料
np_raw[,np_name]

[1] "丹大野生動物重要棲息環境"                            
 [2] "九九峰自然保留區"                                    
 [3] "出雲山自然保留區"                                    
 [4] "利嘉野生動物重要棲息環境"                            
 [5] "北投石自然保留區"                                    
 [6] "十八羅漢山自然保護區"                                
 [7] "南澳闊葉樹林自然保留區"                              
 [8] "台北市中興橋永福橋野生動物重要棲息環境"              
 [9] "台北市野雁保護區"                                    
[10] "台南市四草野生動物保護區"                            
[11] "台南縣曾文溪口北岸黑面琵鷺保護區"                    
[12] "台東紅葉村台東蘇鐵自然保留區"                        
[13] "台東縣海端鄉新武呂溪野生動物重要棲息環境"            
[14] "台東縣海端鄉新武呂溪魚類保護區"                      
[15] "台江國家公園"                                        
[16] "台灣一葉蘭自然保留區"                                
[17] "哈盆自然保留區"                                      
[18] "嘉義縣鰲鼓野生動物重要棲息環境"                      
[19] "坪林台灣油杉自然保留區"                              
[20] "塔山野生動物重要棲息環境"                            
[21] "墾丁國家公園"                                        
[22] "墾丁高位珊瑚礁自然保留區"                            
[23] "壽山國家自然公園"                                    
[24] "大武事業區台灣穗花杉自然保留區"                      
[25] "大武山自然保留區"                                    
[26] "大武臺灣油杉自然保護區"                              
[27] "大肚溪口野生動物保護區"                              
[28] "大肚溪口野生動物重要棲息環境"                        
[29] "太魯閣國家公園"                                      
[30] "宜蘭縣無尾港野生動物重要棲息環境"                    
[31] "宜蘭縣蘭陽溪口野生動物重要棲息環境"                  
[32] "宜蘭縣雙連埤野生動物保護區"                          
[33] "宜蘭縣雙連埤野生動物重要棲息環境"                    
[34] "客雅溪口及香山溼地野生動物重要棲息環境"              
[35] "挖子尾自然保留區"                                    
[36] "插天山自然保留區"                                    
[37] "新竹市濱海野生動物保護區"                            
[38] "旭海-觀音鼻自然保留區"                               
[39] "東沙環礁國家公園"                                    
[40] "桃園觀新藻礁生態系野生動物保護區"                    
[41] "桃園觀新藻礁生態系野生動物重要棲息環境"              
[42] "桃園高榮野生動物保護區"                              
[43] "桃園高榮野生動物重要棲息環境"                        
[44] "棉花嶼、花瓶嶼野生動物保護區"                        
[45] "棉花嶼野生動物重要棲息環境"                          
[46] "棲蘭野生動物重要棲息環境"                            
[47] "櫻花鉤吻鮭野生動物保護區"                            
[48] "水璉野生動物重要棲息環境"                            
[49] "海岸山脈臺東蘇鐵自然保護區"                          
[50] "海岸山脈野生動物重要棲息環境"                        
[51] "浸水營野生動物重要棲息環境"                          
[52] "淡水河紅樹林自然保留區"                              
[53] "澎湖南方四島國家公園"                                
[54] "澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧)"
[55] "澎湖玄武岩自然保留區"                                
[56] "澎湖縣望安島綠蠵龜產卵棲地保護區"                    
[57] "澎湖縣貓嶼海鳥保護區"                                
[58] "澎湖縣貓嶼野生動物重要棲息環境"                      
[59] "烏山頂泥火山地景自然保留區"                          
[60] "烏石鼻海岸自然保留區"                                
[61] "無尾港水鳥保護區"                                    
[62] "玉山國家公園"                                        
[63] "玉里野生動物保護區"                                  
[64] "玉里野生動物重要棲息環境"                            
[65] "瑞岩溪野生動物重要棲息環境"                          
[66] "甲仙四德化石自然保護區"                              
[67] "翡翠水庫食蛇龜野生動物保護區"                        
[68] "翡翠水庫食蛇龜野生動物重要棲息環境"                  
[69] "臺中縣武陵櫻花鉤吻鮭重要棲息環境"                    
[70] "臺中縣高美野生動物重要棲息環境"                      
[71] "臺南市四草野生動物重要棲息環境"                      
[72] "臺南縣曾文溪口黑面琵鷺野生動物重要棲息環境"          
[73] "花瓶嶼野生動物重要棲息環境"                          
[74] "苗栗三義火炎山自然保留區"                            
[75] "茶茶牙賴山野生動物重要棲息環境"                      
[76] "蘭陽溪口水鳥保護區"                                  
[77] "觀霧寬尾鳳蝶野生動物重要棲息環境"                    
[78] "觀音海岸野生動物重要棲息環境"                        
[79] "金門國家公園"                                        
[80] "關山臺灣海棗自然保護區"                              
[81] "關山野生動物重要棲息環境"                            
[82] "關渡自然保留區"                                      
[83] "陽明山國家公園"                                      
[84] "雙鬼湖野生動物重要棲息環境"                

在資料中，因為保護區混有不同類型的保護區，我們希望能夠清楚地依照保護區類型分類，並新增一個欄位註記其保護區屬性。所以我們列出目前有的保護區類別：

1. 國家公園
2. 國家自然公園
3. 自然保留區
4. 野生動物保護區
5. 野生動物重要棲息環境
6. 自然保護區

接下來整理原始資料，看保護區名稱欄位是否有錯誤的地方？並核對[林務局自然保育網](http://conservation.forest.gov.tw/)看資料是否正確

|類別                 | 個數 |  陸域面積 |   海域面積 | 總面積(公告) | 檢核總面積 | 面積差異 |   
|---------------------|------|-----------|------------|--------------|------------|------| 
|自然保留區           | 22   |  65340.81 |     117.10 |   65457.99   |   65457.91 | 0.08 | 
|野生動物保護區       | 20   |  27145.57 |     295.88 |   27441.46   |   27441.45 | 0.01 | 
|野生動物重要棲息環境 | 37   | 325987.02 |     295.88 |  326282.90   |  326282.90 | 0.00 | 
|國家公園             |  9   | 310375.50 |  438573.80 |  748949.30   |  748949.30 | 0.00 | 
|國家自然公園         |  1   |   1122.65 |       0.00 |    1122.65   |    1122.65 | 0.00 | 
|自然保護區           |  6   |  21171.43 |       0.00 |   21171.43   |   21171.43 | 0.00 | 
|總計                 | 95   | 751142.98 |  439282.66 | 1190425.73   | 1190425.64 | 0.09 | 
|扣除重疊面積之總計   |      | 694503.27 |  438986.86 | 1133490.13   | 1133490.13 | 0.00 | 

我們使用 grep 把保護區類型(list: np_type)的數目抓出來，再比對看和自然保育網上的資料是否相同。上表整理自自然保育網上的資料，我們發現陸域加上海域面積和總面積比較，理論上應該要相等，但還是有幾個保護區類型並非相等，可能是因為計算上的誤差導致（最多差到 900 平方公尺)

In [7]:
np_type <- list('國家公園','國家自然公園','自然保留區','野生動物保護區','野生動物重要棲息環境','自然保護區')
get_length <- function(x) {
  f <- length(grep(x, np_raw[, np_name]))
  return(sprintf('%s %s', x, f))
}
lapply(np_type, get_length)
# 總和

[[1]]
[1] "國家公園 9"

[[2]]
[1] "國家自然公園 1"

[[3]]
[1] "自然保留區 22"

[[4]]
[1] "野生動物保護區 12"

[[5]]
[1] "野生動物重要棲息環境 36"

[[6]]
[1] "自然保護區 6"

發現除了「野生動物保護區」和「野生動物重要棲息環境」數值有出入外，保護區總數亦有出入，所以進一步檢查到底是哪裡出了問題。整理網站的表格和我們檢查出來的比對：

| 保護區名稱              | 資料篩選的數量 | 實際數量 |
| ----------------------- | -------------: | -------: |
| 國家公園                |  9             |  9       |  
| 國家自然公園            |  1             |  1       |
| 自然保留區              | 22             | 22       |
| <font color="red">野生動物保護區</font>          | <font color="red">12</font>             | <font color="red">20</font>      |
| <font color="red">野生動物重要棲息環境</font>    | <font color="red">36</font>             | <font color="red">37</font>     |
| 自然保護區              |  6             |  6       |

「野生動物保護區」和「野生動物重要棲息環境」兩者的數量不符，所以使用 [regular expression](https://en.wikipedia.org/wiki/Regular_expression) 搭配 ```grep``` 來找出是哪些保護區名稱和原始資料不同，也就是名稱裡頭沒有「國家公園」、「國家自然公園」、「自然保留區」、「野生動物保護區」、「野生動物重要棲息環境」以及「自然保護區」。

In [8]:
# | 代表 or
np_type_regexp <- '國家公園|國家自然公園|自然保留區|野生動物保護區|野生動物重要棲息環境|自然保護區'
# 反向選擇符合的 pattern
np_raw[grep(np_type_regexp, np_raw[, np_name], invert=TRUE), .(np_name)]

np_name
台北市野雁保護區
台南縣曾文溪口北岸黑面琵鷺保護區
台東縣海端鄉新武呂溪魚類保護區
澎湖縣望安島綠蠵龜產卵棲地保護區
澎湖縣貓嶼海鳥保護區
無尾港水鳥保護區
臺中縣武陵櫻花鉤吻鮭重要棲息環境
蘭陽溪口水鳥保護區
馬祖列島燕鷗保護區


從這裡我們可以得知，不是所有的保護區系統都會有和類型一模一樣的名稱！所以有些保護區並沒有使用「野生動物保護區」，而是用「_動物名稱_」加上「保護區」。此外「野生動物重要棲息環境」也有一個類似的例子。要修正這個問題，我們得先把保護區類型符合「國家公園」、「自然保留區」等資料找出來，並新增一個欄位(np_type)存放其類型，以供後續檢查數目是否和網站上公告的相符。

In [9]:
#先更新保護區類型 (np_type)
put_np_type <- function(x) {
  np_raw[grep(x, np_raw[, np_name]), np_type := x]
}
invisible(lapply(np_type, put_np_type))

Warning message:
In `[.data.table`(np_raw, grep(x, np_raw[, np_name]), `:=`(np_type, : Invalid .internal.selfref detected and fixed by taking a (shallow) copy of the data.table so that := can add this new column by reference. At an earlier point, this data.table has been copied by R (or been created manually using structure() or similar). Avoid key<-, names<- and attr<- which in R currently (and oddly) may copy the whole data.table. Use set* syntax instead to avoid copying: ?set, ?setnames and ?setattr. Also, in R<=v3.0.2, list(DT1,DT2) copied the entire DT1 and DT2 (R's list() used to copy named objects); please upgrade to R>v3.0.2 if that is biting. If this message doesn't help, please report to datatable-help so the root cause can be fixed.

把 np_type 欄位中沒資料的補起來

In [10]:
##  a. 先補野生動物保護區的

# 先找出保護區類型(np_type)欄位為空值的 row number
np_type_is_na <- which(is.na(np_raw[, np_type]))
# 再找出有 np_name 欄位中有「保護區」的 row number
np_type_is_p <- grep('保護區', np_raw[, np_name])
# 兩者取交集，使用 A %in% B
# 使用 set update 資料
set(np_raw, i=np_type_is_na[np_type_is_na %in% np_type_is_p], which(colnames(np_raw) == 'np_type'), '野生動物保護區')

In [11]:
## b. 再補野生動物重要棲息環境

wildlife_imp_env <- which(is.na(np_raw[, np_type]))
set(np_raw, i=wildlife_imp_env, which(colnames(np_raw) == 'np_type'), '野生動物重要棲息環境')

# 再次確認 np_type 是否有空值
#which(is.na(np_raw[, np_type]))

再次檢查看保護區總數和網站公布的資料對不對

In [12]:
# check length of each np_type
get_length_corrected <- function(x) {
  f = length(grep(x, np_raw[, np_type]))
  return(sprintf('%s %s', x, f))
}
lapply(np_type, get_length_corrected)


[[1]]
[1] "國家公園 9"

[[2]]
[1] "國家自然公園 1"

[[3]]
[1] "自然保留區 22"

[[4]]
[1] "野生動物保護區 20"

[[5]]
[1] "野生動物重要棲息環境 37"

[[6]]
[1] "自然保護區 6"

所以到這個步驟確認所有的保護區數量都與網站公布的資料相符合了

*2. 保護區面積*

保護區面積理論上應該都是數值，而且是正整數。所以我們把握住這個原則來檢查資料的正確性

In [13]:
# 看一下面積的資料
np_raw[, area]

[1] "109952"                                     
 [2] "1,198.45"                                   
 [3] "6248.74"                                    
 [4] "1,022.36"                                   
 [5] "0.2"                                        
 [6] "193.01"                                     
 [7] "200"                                        
 [8] "245"                                        
 [9] "245"                                        
[10] "523.848"                                    
[11] "300"                                        
[12] "290.46"                                     
[13] "292"                                        
[14] "292"                                        
[15] "39,310陸域：4,905 海域：34,405"             
[16] "51.89"                                      
[17] "332.7"                                      
[18] "664.48"                                     
[19] "34.6"                                       
[20] "696.38"                                     
[21] "33,288.65　陸域：18,083.50　海域：15,205.15"
[22] "137.625"                                    
[23] "1,123"                                      
[24] "86.4"                                       
[25] "47000"                                      
[26] "5.04"                                       
[27] "2669.73"                                    
[28] "2670"                                       
[29] "92,000"                                     
[30] "103.35"                                     
[31] "206"                                        
[32] "17.1578"                                    
[33] "666"                                        
[34] "1,600.00"                                   
[35] "30"                                         
[36] "7759.17"                                    
[37] "1600"                                       
[38] "841.3"                                      
[39] "353,667.95 陸域：168.97 海域：353,498.98"   
[40] "315"                                        
[41] "315"                                        
[42] "1.11"                                       
[43] "1.11"                                       
[44] "226.38"                                     
[45] "陸域：13.3024 海域：188 總計：201.3024"     
[46] "55991.41"                                   
[47] "7124.7"                                     
[48] "339.86"                                     
[49] "38"                                         
[50] "3,300.59"                                   
[51] "1,119.28"                                   
[52] "76.41"                                      
[53] "35,843.62陸域：370.29海域：35,473.33"       
[54] "176.2544"                                   
[55] "滿潮19.13; 低潮 30.87"                      
[56] "23.33"                                      
[57] "36.2"                                       
[58] "陸域：10.0200 海域：26.1842 總計：36.2042"  
[59] "3.8802"                                     
[60] "311"                                        
[61] "103.35"                                     
[62] "105,490"                                    
[63] "11414.58"                                   
[64] "11414.58"                                   
[65] "2,574.00"                                   
[66] "11.232"                                     
[67] "1295.93"                                    
[68] "1295.93"                                    
[69] "7095"                                       
[70] "701.3"                                      
[71] "523.848"                                    
[72] "634.4344"                                   
[73] "陸域：3.08 海域：22 總計：25.08"            
[74] "219.04"                                     
[75] "2,004.40"                                   
[76] "206"                                        
[77] "23.5"                                       
[78] "519"                                        
[79] "3,719.70"                                   
[80] "54.33"                 

這些資料有什麼問題呢？如果根據之前的定義，面積資料只能有數值，所以把錯誤的地方列出來：

1. 混用數值資料與文字資料<br/>
   例如：'35,843.62陸域：370.29海域：35,473.33'<br/>
   <br/>
   
2. 同一個欄位有兩種不同屬性<br/>
   例如：有的只有單一數值，有的混有陸域和海域面積，有的則是有滿潮和低潮<br/>
   <br/>
   
3. 數值中混有千位數分隔符(comma)<br/>
   例如：'109952' 和 '1<strong><font style="color:red">,</font></strong>198.45'<br/>
   <br/>

4. 精度不一<br/>
   例如：有些數值為小數點下四位、下三位、下兩位，皆無統一標準
   <br/>

所以我們先從比較容易著手的地方更正，先把千位數分隔符去除，再把陸域和海域的面積分拆成兩個欄位。滿潮和低潮資料有點棘手，因為都是屬於陸域的面積範圍，但依照潮汐會有不同的面積大小，因此建議把滿潮和低潮面積加總，把個別的面積放在備註欄內。最後則是統一精度。

In [14]:
# 把千位數的 comma 移除
invisible(np_raw[, area := gsub(',', '', area)])

In [15]:
# 把海域和陸域分開，實作上我們先將 area 轉成 numeric ，如果欄位數值混有文字資料則會變成 NA。然後再把 NA 的 record number 找出來
area_mixed_char_i <- which(is.na(as.numeric(np_raw[, area])))
np_raw[area_mixed_char_i, .(area)]

Warning message:
In which(is.na(as.numeric(np_raw[, area]))): NAs introduced by coercion

area
39310陸域：4905 海域：34405
33288.65 陸域：18083.50 海域：15205.15
353667.95 陸域：168.97 海域：353498.98
陸域：13.3024 海域：188 總計：201.3024
35843.62陸域：370.29海域：35473.33
滿潮19.13; 低潮 30.87
陸域：10.0200 海域：26.1842 總計：36.2042
陸域：3.08 海域：22 總計：25.08
陸域：11.9171 海域：59.6995 總計：71.6166


看起來沒什麼 pattern 可言，所以手動來改吧

In [16]:
terr_area <- list(4905, 18083.5, 168.97, 13.3024, 370.29, 19.13+30.87, 10.02, 3.08, 11.9171)
oceanic_area <- list(34405, 15205.15, 353498.98, 188, 35473.33, 0, 26.1842, 22, 59.6996)

# 先新增一個陸域面積欄位，並把未標註陸域和海域的保護區面積填入
invisible(np_raw[,terr_area := as.numeric(area)])
# 新增海域面積的欄位，預設值為 0
invisible(np_raw[,oceanic_area := 0])

Warning message:
In eval(expr, envir, enclos): NAs introduced by coercion

In [17]:
# 更新陸域面積的資料
set(np_raw, area_mixed_char_i, which(colnames(np_raw) == 'terr_area'), terr_area)
# 更新完之後看對不對
np_raw[area_mixed_char_i, .(np_name, area, terr_area)]

Warning message:
In set(np_raw, area_mixed_char_i, which(colnames(np_raw) == "terr_area"), : Coerced 'list' RHS to 'double' to match the column's type. Either change the target column to 'list' first (by creating a new 'list' vector length 95 (nrows of entire table) and assign that; i.e. 'replace' column), or coerce RHS to 'double' (e.g. 1L, NA_[real|integer]_, as.*, etc) to make your intent clear and for speed. Or, set the column type correctly up front when you create the table and stick to it, please.

np_name,area,terr_area
台江國家公園,39310陸域：4905 海域：34405,4905.0000
墾丁國家公園,33288.65 陸域：18083.50 海域：15205.15,18083.5000
東沙環礁國家公園,353667.95 陸域：168.97 海域：353498.98,168.9700
棉花嶼野生動物重要棲息環境,陸域：13.3024 海域：188 總計：201.3024,13.3024
澎湖南方四島國家公園,35843.62陸域：370.29海域：35473.33,370.2900
澎湖玄武岩自然保留區,滿潮19.13; 低潮 30.87,50.0000
澎湖縣貓嶼野生動物重要棲息環境,陸域：10.0200 海域：26.1842 總計：36.2042,10.0200
花瓶嶼野生動物重要棲息環境,陸域：3.08 海域：22 總計：25.08,3.0800
馬祖列島野生動物重要棲息環境,陸域：11.9171 海域：59.6995 總計：71.6166,11.9171


In [18]:
# 更新海域面積的資料
set(np_raw, area_mixed_char_i, which(colnames(np_raw) == 'oceanic_area'), oceanic_area)
# 更新完之後看對不對
np_raw[area_mixed_char_i, .(np_name, area, oceanic_area)]

Warning message:
In set(np_raw, area_mixed_char_i, which(colnames(np_raw) == "oceanic_area"), : Coerced 'list' RHS to 'double' to match the column's type. Either change the target column to 'list' first (by creating a new 'list' vector length 95 (nrows of entire table) and assign that; i.e. 'replace' column), or coerce RHS to 'double' (e.g. 1L, NA_[real|integer]_, as.*, etc) to make your intent clear and for speed. Or, set the column type correctly up front when you create the table and stick to it, please.

np_name,area,oceanic_area
台江國家公園,39310陸域：4905 海域：34405,34405.0000
墾丁國家公園,33288.65 陸域：18083.50 海域：15205.15,15205.1500
東沙環礁國家公園,353667.95 陸域：168.97 海域：353498.98,353498.9800
棉花嶼野生動物重要棲息環境,陸域：13.3024 海域：188 總計：201.3024,188.0000
澎湖南方四島國家公園,35843.62陸域：370.29海域：35473.33,35473.3300
澎湖玄武岩自然保留區,滿潮19.13; 低潮 30.87,0.0000
澎湖縣貓嶼野生動物重要棲息環境,陸域：10.0200 海域：26.1842 總計：36.2042,26.1842
花瓶嶼野生動物重要棲息環境,陸域：3.08 海域：22 總計：25.08,22.0000
馬祖列島野生動物重要棲息環境,陸域：11.9171 海域：59.6995 總計：71.6166,59.6996


In [19]:
# 確認都沒問題後，將原本 area 內的資料更新 (area = terr_area + oceanic_area) 
invisible(np_raw[,area := terr_area + oceanic_area])

In [20]:
# 記得將澎湖玄武岩自然保留區的滿潮和低潮面積放在備註(note)欄位內
invisible(np_raw[which(np_raw[,np_name]=='澎湖玄武岩自然保留區'), note := '滿潮:19.13;低潮:30.87'])

_3. 地點(location)_

地點應該是文字(character)的資料，所以檢查的重點為內容是否真的為地點？ 例如說這個欄位中應該要顯示縣市、鄉鎮等

In [21]:
# 照例先看一下內容
np_raw[, location]

[1] "國有林林田山事業區第27、28、78-104、118-124林班，木瓜山事業區第48-54、70林班，丹大事業區第1-40林班，巒大事業區第135（第7、10、11、13小班除外）、136-179、181-201林班，濁水溪事業區第15-17、19-21、25-27、30林班"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
 [2] "埔里事業區第8林班30、31小班，第9林班16-19小班，第10林班26、27、30、31、34、35小班，第11林班17-20、23、26-30、32、33小班，第12林班15-20小班，第13林班1、2小班，第15林班1-3、13-18小班，第16林班1、2、5-7小班，第17林班1、2小班，第18林班5-7小班，第19林班5、11、12小班，第20林班22小班"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 [3] "荖濃溪事業區第22-37林班及其外緣之馬里山溪北向、西南向與濁口溪南向、東南向溪山坡各100公尺為界範圍內之土地"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 [4] "國有林台東事業區第7、9、10林班"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [5] "北投溪第2瀧至第4瀧間河堤內的行水區及部分毗鄰河岸地，兩端分別以北投溫泉博物館及熱海飯店前之木棧橋為界"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

看了一下資料，似乎當初切欄位的時候沒有切正確，地點中混有管理機關。檢查了一下應該都是國家公園的跑錯了欄位

In [22]:
np_raw[grep('內政部營建署', np_raw[,location]),]

np_name,area,location,admin_agency,announce_date,np_type,terr_area,oceanic_area,note
台江國家公園,39310.00,內政部營建署 台江國家公園管理處,10/15/1998,,國家公園,4905.00,34405.00,NA
墾丁國家公園,33288.65,內政部營建署 墾丁國家公園管理處,09/01/1971,,國家公園,18083.50,15205.15,NA
太魯閣國家公園,92000.00,內政部營建署 太魯閣國家公園管理處,11/28/1975,,國家公園,92000.00,0.00,NA
東沙環礁國家公園,353667.95,內政部營建署 海洋國家公園管理處,01/17/1996,,國家公園,168.97,353498.98,NA
澎湖南方四島國家公園,35843.62,內政部營建署 海洋國家公園管理處,103/6/8,,國家公園,370.29,35473.33,NA
玉山國家公園,105490.00,內政部營建署 玉山國家公園管理處,04/06/1974,,國家公園,105490.00,0.00,NA
金門國家公園,3719.70,內政部營建署 金門國家公園管理處,10/18/1984,,國家公園,3719.70,0.00,NA
陽明山國家公園,11455.00,內政部營建署 陽明山國家公園管理處,09/01/1974,,國家公園,11455.00,0.00,NA
雪霸國家公園,76850.00,內政部營建署 雪霸國家公園管理處,07/01/1981,,國家公園,76850.00,0.00,NA


不只是位置(location)欄位不對，日期(announce_date)也錯位了，我們一起修正：


In [23]:
national_park_row_id <- grep('內政部營建署', np_raw[,location])
# 要先更正日期，再更新位置才不會覆蓋掉資料
# 把日期更正到正確的位置
set(np_raw, national_park_row_id,
    which(colnames(np_raw)=='announce_date'), 
    np_raw[national_park_row_id, admin_agency])
# 把管理機關更正到正確的位置
set(np_raw, national_park_row_id,
    which(colnames(np_raw)=='admin_agency'),
    np_raw[national_park_row_id, location])
# 把原本範圍(location)的資料刪除
set(np_raw, national_park_row_id,
    which(colnames(np_raw)=='location'), NA
)

# 把國家公園管理處的全形和半形空白移除
set(np_raw, national_park_row_id,
    which(colnames(np_raw)=='admin_agency'),
    gsub('　| ', '', np_raw[national_park_row_id, admin_agency]))

In [24]:
# 再看一下上述更新之後的資料正不正確
np_raw[national_park_row_id,]

np_name,area,location,admin_agency,announce_date,np_type,terr_area,oceanic_area,note
台江國家公園,39310.00,NA,內政部營建署台江國家公園管理處,10/15/1998,國家公園,4905.00,34405.00,NA
墾丁國家公園,33288.65,NA,內政部營建署墾丁國家公園管理處,09/01/1971,國家公園,18083.50,15205.15,NA
太魯閣國家公園,92000.00,NA,內政部營建署太魯閣國家公園管理處,11/28/1975,國家公園,92000.00,0.00,NA
東沙環礁國家公園,353667.95,NA,內政部營建署海洋國家公園管理處,01/17/1996,國家公園,168.97,353498.98,NA
澎湖南方四島國家公園,35843.62,NA,內政部營建署海洋國家公園管理處,103/6/8,國家公園,370.29,35473.33,NA
玉山國家公園,105490.00,NA,內政部營建署玉山國家公園管理處,04/06/1974,國家公園,105490.00,0.00,NA
金門國家公園,3719.70,NA,內政部營建署金門國家公園管理處,10/18/1984,國家公園,3719.70,0.00,NA
陽明山國家公園,11455.00,NA,內政部營建署陽明山國家公園管理處,09/01/1974,國家公園,11455.00,0.00,NA
雪霸國家公園,76850.00,NA,內政部營建署雪霸國家公園管理處,07/01/1981,國家公園,76850.00,0.00,NA


*4. 管理機關(admin_agency)*


In [25]:
# 看一下資料內容
np_raw[, admin_agency]

[1] "行政院農業委員會"                                        
 [2] "行政院農業委員會林務局南投林區管理處(國定自然地景)"      
 [3] "行政院農業委員會林務局屏東林區管理處(國定自然地景)"      
 [4] "行政院農業委員會"                                        
 [5] "臺北市政府產業發展局(市定自然地景)"                      
 [6] ""                                                        
 [7] "行政院農業委員會林務局羅東林區管理處(國定自然地景)"      
 [8] "行政院農委會"                                            
 [9] "行政院農委會"                                            
[10] "行政院農委會"                                            
[11] "行政院農委會"                                            
[12] "行政院農業委員會林務局台東林區管理處(國定自然地景)"      
[13] "行政院農委會"                                            
[14] "行政院農委會"                                            
[15] "內政部營建署台江國家公園管理處"                          
[16] "行政院農業委員會林務局嘉義林區管理處(國定自然地景)"      
[17] "行政院農業委會林業試驗所福山研究中心(國定自然地景)"      
[18] "行政院農業委員會"                                        
[19] "羅東林區管理處(國定自然地景)"                            
[20] "行政院農業委員會"                                        
[21] "內政部營建署墾丁國家公園管理處"                          
[22] "行政院農業委員會林業試驗所恆春分所(國定自然地景)"        
[23] ""                                                        
[24] "行政院農業委員會林務局台東林區管理處(國定自然地景)"      
[25] "行政院農業委員會林務局台東、屏東林區管理處(國定自然地景)"
[26] ""                                                        
[27] "行政院農委會"                                            
[28] "行政院農委會"                                            
[29] "內政部營建署太魯閣國家公園管理處"                        
[30] "行政院農業委員會"                                        
[31] "行政院農委會"                                            
[32] "行政院農業委員會"                                        
[33] "行政院農業委員會"                                        
[34] "行政院農委會"                                            
[35] "新北市政府(國定自然地景)"                                
[36] "行政院農業委員會林務局新竹林區管理處(國定自然地景)"      
[37] "行政院農委會"                                            
[38] "屏東縣政府(縣定自然地景)"                                
[39] "內政部營建署海洋國家公園管理處"                          
[40] "行政院農業委員會"                                        
[41] "行政院農業委員會"                                        
[42] "行政院農業委員會"                                        
[43] "行政院農業委員會"                                        
[44] "行政院農委會"                                            
[45] "行政院農委會"                                            
[46] "行政院農業委員會"                                        
[47] "行政院農委會"                                            
[48] "行政院農業委員會"                                        
[49] ""                                                        
[50] "行政院農業委員會"                                        
[51] "行政院農業委員會"                                        
[52] "行政院農業委員會林務局羅東林區管理處(國定自然地景)"      
[53] "內政部營建署海洋國家公園管理處"                          
[54] "澎湖縣政府(縣定自然地景)"                                
[55] "澎湖縣政府(國定自然地景)"                                
[56] "行政院農委會"                                            
[57] "行政院農委會"                                            
[58] "行政院農委會"                                            
[59] "高雄市政府(國定自然地景)"                                
[60] "行政院農業委員會林務局羅東林區管理處(國定自然地景)"      
[61] "行政院農委會"                                            
[62] "內政部營建署玉山國家公園管理處"                          
[63] "行政院農委會"                                            
[64] "行政院農委會"                                            
[65] "行政院農業委員會"                                        
[66] ""                                                        
[67] "行政院農業委員會"                                        
[68] "行政院農業委員會"                                        
[69] "行政院農委會"                                            
[70] "行政院農業委員會"                                        
[71] "行政院農業委員會"                                        
[72] "行政院農委會"                                            
[73] "行政院農委會"                                            
[74] "行政院農業委員會林務局新竹林區管理處(國定自然地景)"      
[75] "行政院農業委員會"                                     

管理機關的欄位內容中主要有兩種錯誤

1. 機關名稱不統一，又分別有下列兩個子類型錯誤

   1. 同單位，名稱不同
   2. 有些只有一級單位，有些有一級單位及次級單位
   
2. 混有其他資訊

In [26]:
# 先處理自然地景，自然地景有縣定、國定和市定
natural_landscape_row_id = grep('自然地景', np_raw[,admin_agency])
# 使用 regular expression 把自然地景的名稱抓出來，貼到新的欄位 natural_landscape 中
# 另外也把括號刪除 
# todo: 需要再更精緻的 regular expression 作法
invisible(np_raw[natural_landscape_row_id, natural_landscape := 
         gsub('[(,)]', '', gsub('.*[中心|所|局|處|會|府](\\(.*\\))$', '\\1', admin_agency))]
         )
# 把 admin_agency 中原本的自然地景字樣拿掉
set(np_raw, natural_landscape_row_id, which(colnames(np_raw) == 'admin_agency'),
    gsub('\\((.*)\\)$', '', np_raw[natural_landscape_row_id,admin_agency]))

In [27]:
# 再次確認更新的有沒有問題
np_raw[natural_landscape_row_id, .(admin_agency, natural_landscape)]

admin_agency,natural_landscape
行政院農業委員會林務局南投林區管理處,國定自然地景
行政院農業委員會林務局屏東林區管理處,國定自然地景
臺北市政府產業發展局,市定自然地景
行政院農業委員會林務局羅東林區管理處,國定自然地景
行政院農業委員會林務局台東林區管理處,國定自然地景
行政院農業委員會林務局嘉義林區管理處,國定自然地景
行政院農業委會林業試驗所福山研究中心,國定自然地景
羅東林區管理處,國定自然地景
行政院農業委員會林業試驗所恆春分所,國定自然地景
行政院農業委員會林務局台東林區管理處,國定自然地景


其中有個「國定自然自然地景」要修正

In [28]:
set(np_raw, grep('國定自然自然地景', np_raw[, natural_landscape]), which(colnames(np_raw) == 'natural_landscape'),
       '國定自然地景')
# 同樣地，再次確認更新的有沒有問題
np_raw[natural_landscape_row_id, .(admin_agency, natural_landscape)]

admin_agency,natural_landscape
行政院農業委員會林務局南投林區管理處,國定自然地景
行政院農業委員會林務局屏東林區管理處,國定自然地景
臺北市政府產業發展局,市定自然地景
行政院農業委員會林務局羅東林區管理處,國定自然地景
行政院農業委員會林務局台東林區管理處,國定自然地景
行政院農業委員會林務局嘉義林區管理處,國定自然地景
行政院農業委會林業試驗所福山研究中心,國定自然地景
羅東林區管理處,國定自然地景
行政院農業委員會林業試驗所恆春分所,國定自然地景
行政院農業委員會林務局台東林區管理處,國定自然地景


接下來把同單位名稱不同的更正成相同的（例：行政院農委會林務局、行政院農業委員會林務局），先用 unique 看一下有哪些單位

In [29]:
unique(np_raw[,admin_agency])

[1] "行政院農業委員會"                          
 [2] "行政院農業委員會林務局南投林區管理處"      
 [3] "行政院農業委員會林務局屏東林區管理處"      
 [4] "臺北市政府產業發展局"                      
 [5] ""                                          
 [6] "行政院農業委員會林務局羅東林區管理處"      
 [7] "行政院農委會"                              
 [8] "行政院農業委員會林務局台東林區管理處"      
 [9] "內政部營建署台江國家公園管理處"            
[10] "行政院農業委員會林務局嘉義林區管理處"      
[11] "行政院農業委會林業試驗所福山研究中心"      
[12] "羅東林區管理處"                            
[13] "內政部營建署墾丁國家公園管理處"            
[14] "行政院農業委員會林業試驗所恆春分所"        
[15] "行政院農業委員會林務局台東、屏東林區管理處"
[16] "內政部營建署太魯閣國家公園管理處"          
[17] "新北市政府"                                
[18] "行政院農業委員會林務局新竹林區管理處"      
[19] "屏東縣政府"                                
[20] "內政部營建署海洋國家公園管理處"            
[21] "澎湖縣政府"                                
[22] "高雄市政府"                                
[23] "內政部營建署玉山國家公園管理處"            
[24] "內政部營建署金門國家公園管理處"            
[25] "臺北市政府"                                
[26] "內政部營建署陽明山國家公園管理處"          
[27] "內政部營建署雪霸國家公園管理處"            
[28] "行政院國軍退除役官兵輔導委員會森林保育處"

看起來很雜亂，所以我們把農委會各林管處整併成「行政院農業委員會林務局」（包含「行政院農委會林務局」等）。「臺北市政府產業發展局」併至「臺北市政府」、林業試驗所轄下的研究中心整併為「行政院農業委員會林業試驗所」

In [30]:
set(np_raw, grep('行政院農委會|行政院農業委員會|林區管理處', np_raw[, admin_agency]),
            which(colnames(np_raw) == 'admin_agency'),
            '行政院農業委員會林務局')
set(np_raw, grep('行政院農業委員會林業試驗所|林試所|林業試驗所', np_raw[, admin_agency]),
            which(colnames(np_raw) == 'admin_agency'),
            '行政院農業委員會林業試驗所')
set(np_raw, grep('臺北市政府', np_raw[, admin_agency]),
            which(colnames(np_raw) == 'admin_agency'),
            '臺北市政府')

In [31]:
# 再看一次是否正確
unique(np_raw[,admin_agency])

[1] "行政院農業委員會林務局"                  
 [2] "臺北市政府"                              
 [3] ""                                        
 [4] "內政部營建署台江國家公園管理處"          
 [5] "行政院農業委員會林業試驗所"              
 [6] "內政部營建署墾丁國家公園管理處"          
 [7] "內政部營建署太魯閣國家公園管理處"        
 [8] "新北市政府"                              
 [9] "屏東縣政府"                              
[10] "內政部營建署海洋國家公園管理處"          
[11] "澎湖縣政府"                              
[12] "高雄市政府"                              
[13] "內政部營建署玉山國家公園管理處"          
[14] "內政部營建署金門國家公園管理處"          
[15] "內政部營建署陽明山國家公園管理處"        
[16] "內政部營建署雪霸國家公園管理處"          
[17] "行政院國軍退除役官兵輔導委員會森林保育處"

*5. 公告日期(announce date)*

公告日期理論上應該都是日期，所以要注意的就是格式問題，有些資料會混用西元年和中華民國年（簡稱民國年）

In [32]:
# 先看資料的樣子
np_raw[, announce_date]

[1] "89.02.15"                                                                                                                                                                                          
 [2] "89.05.22"                                                                                                                                                                                          
 [3] "81.03.12"                                                                                                                                                                                          
 [4] "89.10.19"                                                                                                                                                                                          
 [5] "臺北市政府產業發展局102年12月26日府產業動字第10233765600號"                                                                                                                                        
 [6] "81年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"                                                                                                                      
 [7] "81.03.12"                                                                                                                                                                                          
 [8] "86.07.31"                                                                                                                                                                                          
 [9] "台北市政府82.11.19日(82)府建三字第82084560號函； 台北市政府83.05.17日(83)府建三字第83027863號函； 台北市政府86.08.15日府建三字第8606078700號公告修正函"                                            
[10] "台南市政府83.11.30日(83)南市建農字第132629號函"                                                                                                                                                    
[11] "台南縣政府91.11.01府農林字0910179659號公告"                                                                                                                                                        
[12] "75.06.27"                                                                                                                                                                                          
[13] "87.11.19"                                                                                                                                                                                          
[14] "台東縣政府87.12.04日87府農林字第87133002號函公告"                                                                                                                                                  
[15] "10/15/1998"                                                                                                                                                                                        
[16] "81.03.12"                                                                                                                                                                                          
[17] "75.06.27"                                                                                                                                                                                          
[18] "98.04.16"                                                                                                                                                                                          
[19] "75.06.27"                                                                                                                                                                                          
[20] "90.05.17"                                                                                                                                                                                          
[21] "09/01/1971"                                                                              

比想像中還要複雜，中間混有日期格式和西元格式，還有公告日期的公文字號等。
處理流程：

1. 先把純日期的資料找出來
2. 把混有日期和公文字號的分開處理
3. 處理超過一筆以上屬性的(也就是保護區有修正過的）

   因為我們的目的是依照年度計算出保護區面積，所以如果公告日期中的屬性超過一筆以上，我們將會重複新增一筆，以日期加上保護區名稱當做是 key。
   例：
   
| np_name                      | announce_date | area      |
|------------------------------|---------------|-----------|
| 宜蘭縣無尾港野生動物重要棲息環境  | 1998-05-22     | 101.6194 |
| 宜蘭縣無尾港野生動物重要棲息環境  | 2014-07-01     | 113.9800 |
| 宜蘭縣無尾港野生動物重要棲息環境  | 2015-06-10     | 103.3500 |


In [33]:
grep_date_1_row_id <- grep('^([0-9]{2,3})\\.([0-9]{2})\\.([0-9]{2})$', np_raw[, announce_date], perl=TRUE)
np_raw[grep_date_1_row_id, announce_date]
set(np_raw, grep_date_1_row_id, which(colnames(np_raw)=='announce_date'),
  gsub('^([0-9]{2,3})\\.([0-9]{2})\\.([0-9]{2})$', '\\1-\\2-\\3', 
       np_raw[grep_date_1_row_id, announce_date], perl=TRUE))
grep_date_2_row_id <- grep('^([0-9]{2,3})-([0-9]{2})-([0-9]{2})$', np_raw[, announce_date], perl=TRUE)

# 把民國年更換成西元年，依照 ISO 格式
yyyy <- as.numeric(format(strptime(np_raw[grep_date_2_row_id, announce_date], format='%Y-%m-%d'), '%Y'))+1911
mmdd <- format(strptime(np_raw[grep_date_2_row_id, announce_date], format='%Y-%m-%d'), '%m-%d')
# 先不用 as.Date，因為其他的資料還沒整理完
invisible(np_raw[grep_date_2_row_id, announce_date:=paste(yyyy,mmdd,sep="-")])


[1] "89.02.15"  "89.05.22"  "81.03.12"  "89.10.19"  "81.03.12"  "86.07.31" 
 [7] "75.06.27"  "87.11.19"  "81.03.12"  "75.06.27"  "98.04.16"  "75.06.27" 
[13] "90.05.17"  "83.01.10"  "75.06.27"  "87.04.07"  "85.07.11"  "83.01.10" 
[19] "81.03.12"  "101.01.20" "103.04.15" "84.06.12"  "89.02.15"  "90.03.13" 
[25] "89.10.19"  "89.10.19"  "75.06.27"  "81.03.12"  "86.04.07"  "83.01.10" 
[31] "89.01.27"  "89.10.19"  "102.12.10" "84.09.23"  "95.12.22"  "91.10.14" 
[37] "84.06.12"  "75.06.27"  "89.10.19"  "89.10.19"  "89.10.19"  "89.02.15" 
[43] "75.06.27"  "89.10.19"  "89.10.19"  "88.12.24"  "87.03.19"  "75.06.27" 
[49] "89.10.19"

In [34]:
# 整理 yyyy/mm/dd 
grep_date_3_row_id <- grep('^([0-9]{1,2})\\/([0-9]{2})\\/([0-9]{1,4})$', np_raw[, announce_date], perl=TRUE)
np_raw[grep_date_3_row_id, .(np_name, announce_date)]

np_name,announce_date
台江國家公園,10/15/1998
墾丁國家公園,09/01/1971
太魯閣國家公園,11/28/1975
東沙環礁國家公園,01/17/1996
玉山國家公園,04/06/1974
金門國家公園,10/18/1984
陽明山國家公園,09/01/1974
雪霸國家公園,07/01/1981


看起來日期不太對，比對[自然保育網的國家公園](http://conservation.forest.gov.tw/nationalpark)，確認應該是 excel 在處理日期時誤把民國年抓成西元了，所以更正如下：

In [35]:
date_3_yyyy <- as.numeric(format(strptime(np_raw[grep_date_3_row_id, announce_date], format='%m/%d/%Y'), '%Y'))-1900+1911
date_3_mmdd <- format(strptime(np_raw[grep_date_3_row_id, announce_date], format='%m/%d/%Y'), 
                     '%m-%d')

invisible(np_raw[grep_date_3_row_id, announce_date := paste(date_3_yyyy,date_3_mmdd,sep="-")])

剩下的手動修正

In [36]:
invisible(np_raw[grep('桃園高榮野生動物重要棲息環境', np_raw[,np_name]), 
       c('announce_date', 'announce_no') := list('2011-12-21','農林務字第1001701346號公告'), with=FALSE])
invisible(np_raw[grep('客雅溪口及香山溼地野生動物重要棲息環境', np_raw[,np_name]), announce_date := '2001-06-08'])
invisible(np_raw[grep('澎湖南方四島國家公園', np_raw[,np_name]), announce_date := '2014-06-08'])

公告日期混用了「民國年」和「公告字號」，我們先把日期僅出現一次的抓出來處理（也就是該保護區公告後沒有其他的修正）

In [37]:
# 使用 stringr::str_count 來計算符合條件的有幾筆
library(stringr)
np_raw[str_count(np_raw[,announce_date], '([0-9]{2,3}\\.[0-9]{2}\\.[0-9]{2})') == 1, .(np_name, announce_date)]

np_name,announce_date
台南市四草野生動物保護區,台南市政府83.11.30日(83)南市建農字第132629號函
台南縣曾文溪口北岸黑面琵鷺保護區,台南縣政府91.11.01府農林字0910179659號公告
台東縣海端鄉新武呂溪魚類保護區,台東縣政府87.12.04日87府農林字第87133002號函公告
宜蘭縣雙連埤野生動物保護區,宜蘭縣政府92.11.07府農畜字0920137729號函公告
桃園觀新藻礁生態系野生動物保護區,桃園縣政府103.07.07日府農植字第1030161774號函公告
桃園高榮野生動物保護區,桃園縣政府101.03.03府農植字第1010041471號公告
棉花嶼、花瓶嶼野生動物保護區,基隆市政府85.03.18日85基府建農字第017128號函公告
櫻花鉤吻鮭野生動物保護區,台中縣政府86.10.01日86府農技字第261771號函公告
海岸山脈臺東蘇鐵自然保護區,70年公告成立國有林自然保護區 95.04.10農林務字第0951700407號公告成立自然保護區
澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧),澎湖縣政府依97.09.23府授農保字第09735010992號函公告 98年9月15日府授農保字第09835011341號函公告修正


看起來還是有例外啊，像是「'70年公告成立國有林自然保護區 95.04.10農林務字第0951700407號公告成立自然保護區'」

In [38]:
# 找出只有出現一次日期的 row id
ann_date_only_once <- which(str_count(np_raw[,announce_date], '([0-9]{2,3}\\.[0-9]{2}\\.[0-9]{2})') == 1)

In [39]:
# 把海岸山脈臺東蘇鐵自然保護區、澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧)的資料從 ann_date_only_once 中移除
ann_date_only_once <- ann_date_only_once[-which(ann_date_only_once == which(np_raw[,np_name]=='海岸山脈臺東蘇鐵自然保護區'))]
ann_date_only_once <- ann_date_only_once[-which(ann_date_only_once == which(np_raw[,np_name]=='澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧)'))]

In [40]:
# 把日期用 regular expression 抓出來
reg.out <- regexpr('[0-9]{2,3}\\.[0-9]{2}\\.[0-9]{2}', np_raw[ann_date_only_once,announce_date])
reg_substr_date <- substr(np_raw[ann_date_only_once, announce_date], reg.out, attr(reg.out, "match.length")+reg.out-1)
# 把 . 取代成 - 以便後續處理
substr_once_date <-  gsub('\\.', '-', reg_substr_date, perl=TRUE)
substr_once_date

[1] "83-11-30"  "91-11-01"  "87-12-04"  "92-11-07"  "103-07-07" "101-03-03"
 [7] "85-03-18"  "86-10-01"  "84-01-17"  "89-01-27"  "93-09-09"  "85-09-16" 
[13] "97-11-28"  "89-01-26"

In [41]:
substr_once_date_ann_no <- gsub('[0-9]{2,3}\\.[0-9]{2}\\.[0-9]{2}', '\\2', np_raw[ann_date_only_once,announce_date])
# 把冗餘的「日」移除
substr_once_date_ann_no <- gsub('日', '', substr_once_date_ann_no)
substr_once_date_ann_no

[1] "台南市政府(83)南市建農字第132629號函"   
 [2] "台南縣政府府農林字0910179659號公告"     
 [3] "台東縣政府87府農林字第87133002號函公告" 
 [4] "宜蘭縣政府府農畜字0920137729號函公告"   
 [5] "桃園縣政府府農植字第1030161774號函公告" 
 [6] "桃園縣政府府農植字第1010041471號公告"   
 [7] "基隆市政府85基府建農字第017128號函公告" 
 [8] "台中縣政府86府農技字第261771號函公告"   
 [9] "澎湖縣政府(84)澎府農漁字第01472號函公告"
[10] "行政院農委會八九農林字第890030020號函"  
[11] "農授林務字第0931700673號"               
[12] "宜蘭縣政府85府農林字第105206號函公告"   
[13] "農林務字第0971701190號公告"             
[14] "連江縣政府八十八連建農字第20084號函"

In [42]:
# 更新 announce_no, announce_date資料
invisible(np_raw[ann_date_only_once, announce_no := substr_once_date_ann_no])
invisible(np_raw[ann_date_only_once, announce_date := substr_once_date])
invisible(np_raw[ann_date_only_once, .(np_name, announce_date, announce_no)])

In [43]:
# 把民國年代換成西元
invisible(np_raw[ann_date_only_once, announce_date :=
          paste(as.numeric(gsub('-[0-9]{2}-[0-9]{2}', '', np_raw[ann_date_only_once, announce_date]))+1911, 
          gsub('^[0-9]{2,3}-', '', np_raw[ann_date_only_once, announce_date]), sep="-")]
        )

接下來處理公告日期有修正過的

In [44]:
ann_date_gt_twice <- which(str_count(np_raw[,announce_date], '([0-9]{2,3}\\.[0-9]{2}\\.[0-9]{2})') > 1)
ann_date_gt_twice

[1]  9 25 27 33 37 57 59 61 91 92

In [45]:
np_raw[ann_date_gt_twice, .(np_name, announce_date)]

np_name,announce_date
台北市野雁保護區,台北市政府82.11.19日(82)府建三字第82084560號函； 台北市政府83.05.17日(83)府建三字第83027863號函； 台北市政府86.08.15日府建三字第8606078700號公告修正函
大武山自然保留區,77.01.13 77.06.08 公告修正
大肚溪口野生動物保護區,84.02.28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」；87.05.22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣雙連埤野生動物重要棲息環境,"92.10.23公告,94.03.21公告修正"
新竹市濱海野生動物保護區,新竹市90.12.14（九十）府建生字第94263號公告； 93.09.23府建生字第0930099959號公告修正。
澎湖縣貓嶼海鳥保護區,澎湖縣政府80.05.24日(80)澎府農漁字第21442號函公告；澎湖縣政府86.04.23日(86)澎府農漁字第22616號公告修正函
烏山頂泥火山地景自然保留區,81.03.12 102.10.18農林務字第1021701051號公告修正範圍
無尾港水鳥保護區,宜蘭縣政府82.09.24. (82)府農林字第106151號公告；宜蘭縣政府87.06.18. 八七府農畜字第64881號公告修正；宜蘭縣政府104.06.23. 府農畜字第1040098949A號公告修正。
高美野生動物保護區,台中縣政府93.09.29府農育字第0930253489-2號台中縣公告高美野生動物保護區、臺中市政府101.06.22府授農林字第10100995651號公告高美野生動物保護區分區管制範圍暨相關管制事項
高雄縣三民鄉楠梓仙溪野生動物保護區,高雄縣政府82.05.26日(82)府農林字第82411號函；高雄縣政府87.04.17日八七府農林字第61413號公告修正函


In [46]:
# 找一下還有哪些 announce_date 是不符合規範的(只能有西元的日期，格式為 YYYY-mm-dd，同筆資料不可以有超過一筆以上的日期)
np_raw[!grep('([0-9]{1,4})-([0-9]{1,2})-([0-9]{1,2})', announce_date, perl=TRUE), .(np_name, announce_date)]

np_name,announce_date
北投石自然保留區,臺北市政府產業發展局102年12月26日府產業動字第10233765600號
十八羅漢山自然保護區,81年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區
台北市野雁保護區,台北市政府82.11.19日(82)府建三字第82084560號函； 台北市政府83.05.17日(83)府建三字第83027863號函； 台北市政府86.08.15日府建三字第8606078700號公告修正函
壽山國家自然公園,98年
大武山自然保留區,77.01.13 77.06.08 公告修正
大武臺灣油杉自然保護區,70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區
大肚溪口野生動物保護區,84.02.28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」；87.05.22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣無尾港野生動物重要棲息環境,87年5月22日公告，103年7月1日公告修正，104年6月10日再度公告修正。
宜蘭縣雙連埤野生動物重要棲息環境,"92.10.23公告,94.03.21公告修正"
新竹市濱海野生動物保護區,新竹市90.12.14（九十）府建生字第94263號公告； 93.09.23府建生字第0930099959號公告修正。


公告日期的資料真的很零亂，剩下還未處理的日期有兩種以上的 pattern

1. 民國年月日，用國字的「年」、「月」、「日」，例如：80年5月24日
2. 民國年月日，用「.」來分隔，例如： 80.05.24
    1. 有的後面還會加上「日」 
3. 僅有民國年<br/>
   例：「81年」公告成立...

In [47]:
# 1. 處理有民國年、月、日的資料
# 先把 x 年 y 月 n 日 取代成 x.y.z

invisible(np_raw[grep('([0-9]{2,3})年([0-9]{1,2})月([0-9]{1,2})日', announce_date), 
       announce_date := gsub('([0-9]{2,3})年([0-9]{1,2})月([0-9]{1,2})日', 
                            '\\1.\\2.\\3', announce_date)]
          )

In [48]:
# 再看一下剩下的 annnounce_date 有哪些特徵
np_raw[grep('[0-9]{2,4}-[0-9]{2}-[0-9]{2}',announce_date, invert = TRUE, perl=TRUE), announce_date]

[1] "臺北市政府產業發展局102.12.26府產業動字第10233765600號"                                                                                                                                            
 [2] "81年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"                                                                                                                      
 [3] "台北市政府82.11.19日(82)府建三字第82084560號函； 台北市政府83.05.17日(83)府建三字第83027863號函； 台北市政府86.08.15日府建三字第8606078700號公告修正函"                                            
 [4] "98年"                                                                                                                                                                                              
 [5] "77.01.13 77.06.08 公告修正"                                                                                                                                                                        
 [6] "70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"                                                                                                                      
 [7] "84.02.28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」；87.05.22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。"
 [8] "87.5.22公告，103.7.1公告修正，104.6.10再度公告修正。"                                                                                                                                              
 [9] "92.10.23公告,94.03.21公告修正"                                                                                                                                                                     
[10] "新竹市90.12.14（九十）府建生字第94263號公告； 93.09.23府建生字第0930099959號公告修正。"                                                                                                            
[11] "70年公告成立國有林自然保護區 95.04.10農林務字第0951700407號公告成立自然保護區"                                                                                                                     
[12] "澎湖縣政府依97.09.23府授農保字第09735010992號函公告 98.9.15府授農保字第09835011341號函公告修正"                                                                                                    
[13] "澎湖縣政府80.05.24日(80)澎府農漁字第21442號函公告；澎湖縣政府86.04.23日(86)澎府農漁字第22616號公告修正函"                                                                                          
[14] "81.03.12 102.10.18農林務字第1021701051號公告修正範圍"                                                                                                                                              
[15] "宜蘭縣政府82.09.24. (82)府農林字第106151號公告；宜蘭縣政府87.06.18. 八七府農畜字第64881號公告修正；宜蘭縣政府104.06.23. 府農畜字第1040098949A號公告修正。"                                         
[16] "80年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"                                                                                                                      
[17] "行政院農業委員會102.12.10農林務字第1021701271號"                                                                                                                                                   
[18] "70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"                                                                                                                      
[19] "70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"                                                                                                                      
[20] "台中縣政府93.09.29府農育字第0930253489-2號台中縣公告高美野生動物保護區、臺中市政府101.06.22府授農林字第10100995651號公告高美野生動物保護區分區管制範圍暨相關管制事項"                              
[21] "高雄縣政府82.05.26日(82)府農林字第82411號函；高雄縣政府87.04.17日八七府農林字第61413號公告修正函"

剩下的 announce_date 欄位資料不符合規範的，先把只有「民國年」的抓出來。先將這些只有「民國年」的資料用 regular expression 撈出來看看是不是還有什麼其他的 pattern:

In [49]:
np_raw[grep('[0-9]{2}年', announce_date), announce_date]

[1] "81年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區" 
[2] "98年"                                                                         
[3] "70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區" 
[4] "70年公告成立國有林自然保護區 95.04.10農林務字第0951700407號公告成立自然保護區"
[5] "80年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區" 
[6] "70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區" 
[7] "70年公告成立國有林自然保護區 95.4.10農林務字第0951700407號公告成立自然保護區"

看起來 pattern 可以一次到位來處理，為了接續能夠正常處理日期格式，我們把只有「民國年」的字串統一改成西元年-01-01 (ex: 2016-01-01)，同樣也是使用 regular expression 來取代。另外也一起把 「民國年.月.日」取代成 「民國年-月-日」(ex: 81.01.01 -> 81-01-01)

In [50]:
invisible(np_raw[grep('([0-9]{2})年', announce_date), 
       announce_date:= gsub('([0-9]{2})年', '\\1-01-01', announce_date, perl=TRUE)]
          )
invisible(np_raw[grep('([0-9]{2,3}\\.[0-9]{1,2}\\.[0-9]{1,2})', announce_date, perl=TRUE), 
       announce_date:= gsub('([0-9]{2,3})\\.([0-9]{1,2})\\.([0-9]{1,2})', '\\1-\\2-\\3', announce_date, perl=TRUE)]
          )

In [51]:
np_raw[grep('([0-9]{4}-[0-9]{2}-[0-9]{2})',announce_date, invert = TRUE, perl=TRUE), .(np_name, announce_date)]

np_name,announce_date
北投石自然保留區,臺北市政府產業發展局102-12-26府產業動字第10233765600號
十八羅漢山自然保護區,81-01-01公告成立國有林自然保護區 95-4-10農林務字第0951700407號公告成立自然保護區
台北市野雁保護區,台北市政府82-11-19日(82)府建三字第82084560號函； 台北市政府83-05-17日(83)府建三字第83027863號函； 台北市政府86-08-15日府建三字第8606078700號公告修正函
壽山國家自然公園,98-01-01
大武山自然保留區,77-01-13 77-06-08 公告修正
大武臺灣油杉自然保護區,70-01-01公告成立國有林自然保護區 95-4-10農林務字第0951700407號公告成立自然保護區
大肚溪口野生動物保護區,84-02-28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」；87-05-22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣無尾港野生動物重要棲息環境,87-5-22公告，103-7-1公告修正，104-6-10再度公告修正。
宜蘭縣雙連埤野生動物重要棲息環境,"92-10-23公告,94-03-21公告修正"
新竹市濱海野生動物保護區,新竹市90-12-14（九十）府建生字第94263號公告； 93-09-23府建生字第0930099959號公告修正。


把日期統一後(即：民國年-月-日)，接下來再繼續把 announce_date 中出現兩筆以上的日期資料列找出來

In [52]:
np_raw[which(str_count(np_raw[,announce_date], '([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})') > 1), .(np_name, announce_date)]

np_name,announce_date
十八羅漢山自然保護區,81-01-01公告成立國有林自然保護區 95-4-10農林務字第0951700407號公告成立自然保護區
台北市野雁保護區,台北市政府82-11-19日(82)府建三字第82084560號函； 台北市政府83-05-17日(83)府建三字第83027863號函； 台北市政府86-08-15日府建三字第8606078700號公告修正函
大武山自然保留區,77-01-13 77-06-08 公告修正
大武臺灣油杉自然保護區,70-01-01公告成立國有林自然保護區 95-4-10農林務字第0951700407號公告成立自然保護區
大肚溪口野生動物保護區,84-02-28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」；87-05-22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣無尾港野生動物重要棲息環境,87-5-22公告，103-7-1公告修正，104-6-10再度公告修正。
宜蘭縣雙連埤野生動物重要棲息環境,"92-10-23公告,94-03-21公告修正"
新竹市濱海野生動物保護區,新竹市90-12-14（九十）府建生字第94263號公告； 93-09-23府建生字第0930099959號公告修正。
海岸山脈臺東蘇鐵自然保護區,70-01-01公告成立國有林自然保護區 95-04-10農林務字第0951700407號公告成立自然保護區
澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧),澎湖縣政府依97-09-23府授農保字第09735010992號函公告 98-9-15府授農保字第09835011341號函公告修正


看起來好糟糕，分隔不同公告修正日期使用的特徵有：全形「；」分隔、空白、全形「，」、半形「,」。該怎麼處理比較好？逐一檢視，同一列通常會使用相同的分隔符號。所以我們再使用 regular expression 來統一分隔符號為「|」(pipeline)

In [53]:
invisible(np_raw[which(str_count(np_raw[,announce_date], '([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})') > 1), 
       announce_date := gsub('[；，\\, ]', '|', perl=TRUE, announce_date)]
          )

In [54]:
# 看一下取代的正不正確
np_raw[which(str_count(np_raw[,announce_date], '([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})') > 1), .(np_name, announce_date)]

np_name,announce_date
十八羅漢山自然保護區,81-01-01公告成立國有林自然保護區|95-4-10農林務字第0951700407號公告成立自然保護區
台北市野雁保護區,台北市政府82-11-19日(82)府建三字第82084560號函||台北市政府83-05-17日(83)府建三字第83027863號函||台北市政府86-08-15日府建三字第8606078700號公告修正函
大武山自然保留區,77-01-13|77-06-08|公告修正
大武臺灣油杉自然保護區,70-01-01公告成立國有林自然保護區|95-4-10農林務字第0951700407號公告成立自然保護區
大肚溪口野生動物保護區,84-02-28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」|87-05-22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣無尾港野生動物重要棲息環境,87-5-22公告|103-7-1公告修正|104-6-10再度公告修正。
宜蘭縣雙連埤野生動物重要棲息環境,92-10-23公告|94-03-21公告修正
新竹市濱海野生動物保護區,新竹市90-12-14（九十）府建生字第94263號公告||93-09-23府建生字第0930099959號公告修正。
海岸山脈臺東蘇鐵自然保護區,70-01-01公告成立國有林自然保護區|95-04-10農林務字第0951700407號公告成立自然保護區
澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧),澎湖縣政府依97-09-23府授農保字第09735010992號函公告|98-9-15府授農保字第09835011341號函公告修正


資料看起來愈來愈乾淨了！但是還是有一些細部需要修正，例如:

分隔看起來不正確的，有些應該是分隔符多了空白<br>
   1. 77-01-13|77-06-08|公告修正 (後面 77-06-08 公告修正中間分隔符號應該去掉)
   2. 台北市政府82-11-19日(82)府建三字第82084560號函||台北市政府83-05-17日(83)府建三字第83027863號函 ... (兩個 |)

因此我們再來手動修正

In [55]:
# 取代 double pipeline
invisible(np_raw[which(str_count(np_raw[,announce_date], '([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})') > 1), 
       announce_date := gsub('\\|\\|', '\\|', perl=TRUE, announce_date)]
          )

In [56]:
# 看一下取代的正不正確
np_raw[which(str_count(np_raw[,announce_date], '([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})') > 1), .(np_name, announce_date)]

np_name,announce_date
十八羅漢山自然保護區,81-01-01公告成立國有林自然保護區|95-4-10農林務字第0951700407號公告成立自然保護區
台北市野雁保護區,台北市政府82-11-19日(82)府建三字第82084560號函|台北市政府83-05-17日(83)府建三字第83027863號函|台北市政府86-08-15日府建三字第8606078700號公告修正函
大武山自然保留區,77-01-13|77-06-08|公告修正
大武臺灣油杉自然保護區,70-01-01公告成立國有林自然保護區|95-4-10農林務字第0951700407號公告成立自然保護區
大肚溪口野生動物保護區,84-02-28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」|87-05-22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣無尾港野生動物重要棲息環境,87-5-22公告|103-7-1公告修正|104-6-10再度公告修正。
宜蘭縣雙連埤野生動物重要棲息環境,92-10-23公告|94-03-21公告修正
新竹市濱海野生動物保護區,新竹市90-12-14（九十）府建生字第94263號公告|93-09-23府建生字第0930099959號公告修正。
海岸山脈臺東蘇鐵自然保護區,70-01-01公告成立國有林自然保護區|95-04-10農林務字第0951700407號公告成立自然保護區
澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧),澎湖縣政府依97-09-23府授農保字第09735010992號函公告|98-9-15府授農保字第09835011341號函公告修正


剩下不正確分隔的「大武山自然保留區」、「無尾港水鳥保護區」以及「高美野生動物保護區」(這個是全形頓號(、)，漏掉了)再手動來更新

In [57]:
invisible(np_raw[grep('大武山自然保留區', np_name), 
       announce_date := '77-01-13|77-06-08公告修正']
)
invisible(np_raw[grep('無尾港水鳥保護區', np_name), 
       announce_date := '宜蘭縣政府82-09-24(82)府農林字第106151號公告|宜蘭縣政府87-06-18八七府農畜字第64881號公告修正|宜蘭縣政府104-06-23府農畜字第1040098949A號公告修正。']                
)
invisible(np_raw[grep('高美野生動物保護區', np_name),
       announce_date := '台中縣政府93-09-29府農育字第0930253489-2號台中縣公告高美野生動物保護區|臺中市政府101-06-22府授農林字第10100995651號公告高美野生動物保護區分區管制範圍暨相關管制事項']
)

接下來得想一下怎麼把同一個保護區內，有修正公告的分開變成不同的資料列

In [58]:
np_raw[which(str_count(np_raw[,announce_date], '([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})') > 1), .(np_name, announce_date)]

np_name,announce_date
十八羅漢山自然保護區,81-01-01公告成立國有林自然保護區|95-4-10農林務字第0951700407號公告成立自然保護區
台北市野雁保護區,台北市政府82-11-19日(82)府建三字第82084560號函|台北市政府83-05-17日(83)府建三字第83027863號函|台北市政府86-08-15日府建三字第8606078700號公告修正函
大武山自然保留區,77-01-13|77-06-08公告修正
大武臺灣油杉自然保護區,70-01-01公告成立國有林自然保護區|95-4-10農林務字第0951700407號公告成立自然保護區
大肚溪口野生動物保護區,84-02-28日彰化縣政府(84)彰府農林字第33474號函暨台中縣政府(84)府農技字第04512號函公告為「大肚溪口水鳥保護區」|87-05-22日彰化縣政府八七彰府農林字第090660號函公告修正為「大肚溪口野生動物保護區」。
宜蘭縣無尾港野生動物重要棲息環境,87-5-22公告|103-7-1公告修正|104-6-10再度公告修正。
宜蘭縣雙連埤野生動物重要棲息環境,92-10-23公告|94-03-21公告修正
新竹市濱海野生動物保護區,新竹市90-12-14（九十）府建生字第94263號公告|93-09-23府建生字第0930099959號公告修正。
海岸山脈臺東蘇鐵自然保護區,70-01-01公告成立國有林自然保護區|95-04-10農林務字第0951700407號公告成立自然保護區
澎湖南海玄武岩自然保留區(東吉嶼、西吉嶼、頭巾、鐵砧),澎湖縣政府依97-09-23府授農保字第09735010992號函公告|98-9-15府授農保字第09835011341號函公告修正


In [59]:
# 把 announce_date 欄位中的屬性依照 | (pipeline)分開先使用 sptrsplit，
# 會得到 list('a1', 'a2', 'a2')，把它 unlist 之後再用 list() 整理，依照其他的欄位當成 key
np_sorted <- np_raw[, list(announce_date = unlist(strsplit(announce_date, '\\|'))), 
                 by=c('np_type', 'np_name', 'area', 'terr_area', 'oceanic_area', 
                      'location', 'admin_agency', 'natural_landscape', 'announce_no', 'note')]

In [60]:
# 根據保護區名稱整理資料，再次確認
np_sorted[, by=c('np_name')]

np_type,np_name,area,terr_area,oceanic_area,location,admin_agency,natural_landscape,announce_no,note,announce_date
野生動物重要棲息環境,丹大野生動物重要棲息環境,109952.000,109952.000,0.00,國有林林田山事業區第27、28、78-104、118-124林班，木瓜山事業區第48-54、70林班，丹大事業區第1-40林班，巒大事業區第135（第7、10、11、13小班除外）、136-179、181-201林班，濁水溪事業區第15-17、19-21、25-27、30林班,行政院農業委員會林務局,NA,NA,NA,2000-02-15
自然保留區,九九峰自然保留區,1198.450,1198.450,0.00,埔里事業區第8林班30、31小班，第9林班16-19小班，第10林班26、27、30、31、34、35小班，第11林班17-20、23、26-30、32、33小班，第12林班15-20小班，第13林班1、2小班，第15林班1-3、13-18小班，第16林班1、2、5-7小班，第17林班1、2小班，第18林班5-7小班，第19林班5、11、12小班，第20林班22小班,行政院農業委員會林務局,國定自然地景,NA,NA,2000-05-22
自然保留區,出雲山自然保留區,6248.740,6248.740,0.00,荖濃溪事業區第22-37林班及其外緣之馬里山溪北向、西南向與濁口溪南向、東南向溪山坡各100公尺為界範圍內之土地,行政院農業委員會林務局,國定自然地景,NA,NA,1992-03-12
野生動物重要棲息環境,利嘉野生動物重要棲息環境,1022.360,1022.360,0.00,國有林台東事業區第7、9、10林班,行政院農業委員會林務局,NA,NA,NA,2000-10-19
自然保留區,北投石自然保留區,0.200,0.200,0.00,北投溪第2瀧至第4瀧間河堤內的行水區及部分毗鄰河岸地，兩端分別以北投溫泉博物館及熱海飯店前之木棧橋為界,臺北市政府,市定自然地景,NA,NA,臺北市政府產業發展局102-12-26府產業動字第10233765600號
自然保護區,十八羅漢山自然保護區,193.010,193.010,0.00,旗山事業區第55林班之一部分，西與旗山事業區第49、50林班為界，南至新威村，北與六龜區義寶村、文武村為鄰。,,NA,NA,NA,81-01-01公告成立國有林自然保護區
自然保護區,十八羅漢山自然保護區,193.010,193.010,0.00,旗山事業區第55林班之一部分，西與旗山事業區第49、50林班為界，南至新威村，北與六龜區義寶村、文武村為鄰。,,NA,NA,NA,95-4-10農林務字第0951700407號公告成立自然保護區
自然保留區,南澳闊葉樹林自然保留區,200.000,200.000,0.00,宜蘭縣南澳鄉羅東林區管理處和平事業區第 87 林班第1-6 小班,行政院農業委員會林務局,國定自然地景,NA,NA,1992-03-12
野生動物重要棲息環境,台北市中興橋永福橋野生動物重要棲息環境,245.000,245.000,0.00,中興橋至永福橋間低水護岸起至縣市界止之河域及光復橋上游六百公尺高灘地,行政院農業委員會林務局,NA,NA,NA,1997-07-31
野生動物保護區,台北市野雁保護區,245.000,245.000,0.00,淡水河流域大漢溪與新店溪交界處，北起中興橋，南至永福橋間，東以台北市萬華區的河濱公園外側低水護岸為界，西至(水域間)臺北市與新北市界線。本保護區所轄之範圍 (為區內所有的草澤、泥灘、水域，及光復橋上游600 公尺之高灘地,行政院農業委員會林務局,NA,NA,NA,台北市政府82-11-19日(82)府建三字第82084560號函


In [61]:
# 更新 announce_no (把 announce_date 中的民國年找出來，再把日期移掉更新資料至 announce_no 中)
# 作法：反選擇 announce_date 中的西元年日期，例如 2016-08-15

# 把日期從公告文號中除去
invisible(np_sorted[grep('([0-9]{4}-[0-9]{1,2}-[0-9]{1,2})', announce_date, invert = TRUE, perl=TRUE), 
          announce_no := gsub('([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})', '\\2', announce_date)]
          )

invisible(np_sorted[grep('(^[0-9]{4}-[0-9]{1,2}-[0-9]{1,2})', announce_date, invert = TRUE, perl=TRUE), 
          announce_date := gsub('^([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})(.*)',  '\\1', announce_date, perl=TRUE)]
          )

invisible(np_sorted[grep('[縣|會|府|公告]', announce_date, perl=TRUE),
          announce_date := gsub('(.*)([0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2})(.*)',  '\\2', announce_date, perl=TRUE)]
          )

In [62]:
# bug: 有些民國超過 100 年的 1 會被去除，所以把它加回來
invisible(np_sorted[grep('(^0[1-9]-[0-9]{1,2}-[0-9]{1,2})(.*)', announce_date, perl=TRUE), 
          announce_date := gsub('(^0[1-9])-([0-9]{1,2})-([0-9]{1,2})','1\\1-\\2-\\3', announce_date)])

In [63]:
# 把最後的民國年找出來，並換算成西元的
roc_year_rowid <- grep('^[0-9]{2,3}-[0-9]{1,2}-[0-9]{1,2}$', np_sorted[, announce_date], perl=TRUE)
fmt_yr <- np_sorted[roc_year_rowid, as.numeric(format(strptime(announce_date, '%Y-%m-%d'), '%Y'))+1911]
fmt_mm_dd <- np_sorted[roc_year_rowid, format(strptime(announce_date, '%Y-%m-%d'), '%m-%d')]
invisible(np_sorted[roc_year_rowid, announce_date := paste(fmt_yr, fmt_mm_dd, sep="-")])

到這裡清理資料已告一段落，接下來就是把資料補齊以及再次檢查資料內容完整性

1. 壽山國家自然公園的資料根據網站更新面積、位置範圍
2. 各個保護區的歷史資料(面積、範圍)核對
3. 各欄位定義以及完整性的再次檢查


In [64]:
# 更新壽山國家自然公園
# 壽山928.714公頃（排除桃源里舊聚落、中山大學及台泥私有地）、半屏山163.3公頃（包括山麓園滯洪沈砂池）、大小龜山及鳳山縣舊城遺址19.39公頃、旗後山11.25公頃
invisible(np_sorted[np_name=='壽山國家自然公園', 
          location := '壽山928.714公頃（排除桃源里舊聚落、中山大學及台泥私有地）、半屏山163.3公頃（包括山麓園滯洪沈砂池）、大小龜山及鳳山縣舊城遺址19.39公頃、旗後山11.25公頃',
    ])
set(np_sorted, which(np_sorted[,np_name=='壽山國家自然公園']),
    which(colnames(np_sorted) %in% c('terr_area', 'area', 'admin_agency', 'announce_date')), 
    list(1122.654, 1122.654, '內政部營建署壽山國家自然公園管理處', '2011-11-01'))

Warning message:
In bmerge(i, x, leftcols, rightcols, io <- FALSE, xo, roll = 0, : A known encoding (latin1 or UTF-8) was detected in a join column. data.table compares the bytes currently, so doesn't support *mixed* encodings well; i.e., using both latin1 and UTF-8, or if any unknown encodings are non-ascii and some of those are marked known and others not. But if either latin1 or UTF-8 is used exclusively, and all unknown encodings are ascii, then the result should be ok. In future we will check for you and avoid this warning if everything is ok. The tricky part is doing this without impacting performance for ascii-only cases.

In [65]:
# 檢查 announce date 數值是否都正確
np_sorted[,announce_date]

[1] "2000-02-15" "2000-05-22" "1992-03-12" "2000-10-19" "2013-12-26"
  [6] "1992-01-01" "2006-04-10" "1992-03-12" "1997-07-31" "1993-11-19"
 [11] "1994-05-17" "1997-08-15" "1994-11-30" "2002-11-01" "1986-06-27"
 [16] "1998-11-19" "1998-12-04" "2009-10-15" "1992-03-12" "1986-06-27"
 [21] "2009-04-16" "1986-06-27" "2001-05-17" "1982-09-01" "1994-01-10"
 [26] "2011-11-01" "1986-06-27" "1988-01-13" "1988-06-08" "1981-01-01"
 [31] "2006-04-10" "1995-02-28" "1998-05-22" "1998-04-07" "1986-11-28"
 [36] "1998-05-22" "2014-07-01" "2015-06-10" "1996-07-11" "2003-11-07"
 [41] "2003-10-23" "2005-03-21" "2001-06-08" "1994-01-10" "1992-03-12"
 [46] "2001-12-14" "2004-09-23" "2012-01-20" "2007-01-17" "2014-07-07"
 [51] "2014-04-15" "2012-03-03" "2011-12-21" "1996-03-18" "1995-06-12"
 [56] "2000-02-15" "1997-10-01" "2001-03-13" "1981-01-01" "2006-04-10"
 [61] "2000-10-19" "2000-10-19" "1986-06-27" "2014-06-08" "2008-09-23"
 [66] "2009-09-15" "1992-03-12" "1995-01-17" "1991-05-24" "1997-04-23"
 [71] "1997-04-07" "1992-03-12" "2013-10-18" "1994-01-10" "1993-09-24"
 [76] "1998-06-18" "2015-06-23" "1985-04-06" "2000-01-27" "2000-01-27"
 [81] "2000-10-19" "1991-01-01" "2006-04-10" "2013-12-10" "2013-12-10"
 [86] "1995-09-23" "2004-09-09" "2006-12-22" "2002-10-14" "1995-06-12"
 [91] "1986-06-27" "2000-10-19" "1996-09-16" "2000-10-19" "2000-10-19"
 [96] "1995-10-18" "1981-01-01" "2006-04-10" "2000-02-15" "1986-06-27"
[101] "1985-09-01" "2000-10-19" "2000-10-19" "1992-07-01" "1981-01-01"
[106] "2006-04-10" "2008-11-28" "2000-01-26" "1999-12-24" "2004-09-29"
[111] "2012-06-22" "1993-05-26" "1998-04-17" "1998-03-19" "1986-06-27"
[116] "2000-10-19"

看起來應該沒有什麼太大的問題，接下來則手動檢查自然保育網上面的公文掃描檔資料，把自然保護區的歷史面積資料更新

**空間資料重疊的扣除**

原始資料收集：

* [野生動物保護區 ESRI Shapefile](http://data.gov.tw/node/25540)
* [野生動物重要棲息環境 ESRI Shapefile](http://data.gov.tw/node/9932)
* [自然保留區 ESRI Shapefile](http://data.gov.tw/node/9933)
* [國家公園公告面積](http://statis.moi.gov.tw/micst/~w105101315504168017633021-c0820101.csv)


In [ ]:
np_sorted

## 參考文獻及延伸閱讀

* Chapman, A. D. (2005a). Principles of data quality. Report for the Global Biodiversity Information Facility 2004, Copenhagen. URL: http://www.gbif.org/prog/digit/data_quality/data_quality
* Chapman, A. D. (2005b). Principles and methods of data cleaning—Primary Species and Species-Occurrence Data, version 1.0. Report for the Global Biodiversity Information Facility 2005, Copenhagen.
* Maletic, J. I., & Marcus, A. (2000). Data Cleansing: Beyond Integrity Analysis. Proceedings of the Conference on Information Quality.

* [壽山國家自然公園管理處-本處沿革](http://snnp.cpami.gov.tw/chinese/index.php?option=com_content&view=article&id=6&Itemid=173)
* [行政院農業委員會林務局自然保育網](http://conservation.forest.gov.tw)